We continued to compare the differences between single feature models and multiple feature models.

Here we tried to compare single linreg with string elastic net to find some
additional transcripts that explain protein level.

In [10]:
data_dir = '/Volumes/Ginkgo_Data/Data/2021-11_CPTACTrainedModels/out2'
feature = 'string' # 'corum' #'string'
out_dir = f'res2_{feature}_coefs'

In [11]:
import os
import pickle
from predict_protein import download_cptac, select_features, train_model, utils
import pandas as pd

cptac_list = download_cptac.download_cptac(n_tumors=8)
tumor_df = pd.concat(cptac_list)

os.makedirs(out_dir, exist_ok=True)

cptac warning: transcriptomics data was not found for the following samples, so transcriptomics data columns were filled with NaN for these samples: C3L-00563.N, C3L-00605.N, C3L-00769.N, C3L-00770.N, C3L-00771.N, C3L-00930.N, C3L-00947.N, C3L-00963.N, C3L-01246.N, C3L-01249.N, C3L-01252.N, C3L-01256.N, C3L-01257.N, C3L-01744.N, C3N-00200.N, C3N-00729.N, C3N-01211.N, NX1.N, NX10.N, NX11.N, NX12.N, NX13.N, NX14.N, NX15.N, NX16.N, NX17.N, NX18.N, NX2.N, NX3.N, NX4.N, NX5.N, NX6.N, NX7.N, NX8.N, NX9.N (/Users/edwardlau/PythonProjects/CPTAC_Protein/predict_protein/download_cptac.py, line 37)
cptac warning: proteomics data was not found for the following samples, so proteomics data columns were filled with NaN for these samples: 01OV002, 01OV008, 01OV010, 01OV013, 01OV019, 01OV045, 01OV046, 01OV049, 02OV035, 02OV040, 02OV042, 02OV045, 04OV041, 11OV009, 13OV004, 17OV019, 17OV034, 22OV001, 26OV010 (/Users/edwardlau/PythonProjects/CPTAC_Protein/predict_protein/download_cptac.py, line 41)
cptac

In [ ]:
tm = train_model.LearnCPTAC(tumor_df)
tm.included_features = feature

In [ ]:
get_dataframe = utils.get_dataframe


linreg = get_dataframe(os.path.join(data_dir, f'tumor8_linreg_single.p'))
linreg = linreg.rename(columns={'corr_test': 'linreg_corr'})

elastic = get_dataframe(os.path.join(data_dir, f'tumor8_elastic_{feature}.p'))
elastic = elastic.rename(columns={'corr_test': 'elastic_corr'})


In [ ]:
elastic_cf = elastic[['elastic_corr', 'num_obs', 'num_features']].merge(linreg[['linreg_corr']], left_index=True, right_index=True)
elastic_cf['diff'] = elastic_cf.elastic_corr - elastic_cf.linreg_corr
proteins_of_interest = list(elastic_cf[(elastic_cf.num_features > 1) & (elastic_cf['diff'] > 0.33)].index)

In [ ]:
with open(os.path.join(data_dir, f'tumor8_elastic_{feature}.p'), 'rb') as f:
    results = pickle.load(f)

for protein_of_interest in proteins_of_interest:
    # with open('../out2/tumor8_linreg_single.p', 'rb') as f:
    #     results = pickle.load(f)
    #     linreg_model  = [result['model'] for result in results if result['metrics'].index[0] == protein_of_interest][0]

    elastic_model  = [result['model'] for result in results if result['metrics'].index[0] == protein_of_interest][0]

    # x_train, x_test, y_train, y_test = tm.get_train_test(protein_to_do=protein_of_interest)

    elastic_coefs = pd.DataFrame(list(zip(elastic_model.feature_names_in_,
                                      elastic_model.coef_)),
               columns =['Transcript', 'Coef'])
    elastic_coefs = elastic_coefs.sort_values(by='Coef')
    elastic_coefs.to_csv(os.path.join(out_dir, f'{protein_of_interest}_ElasticNet.csv'))

In [ ]:
with open(os.path.join(data_dir, 'tumor8_forest_corum.p'), 'rb') as f:
    results = pickle.load(f)

for protein_of_interest in proteins_of_interest:

    forest_model  = [result['model'] for result in results if result['metrics'].index[0] == protein_of_interest][0]

    forest_importance = pd.DataFrame(list(zip(forest_model.feature_names_in_,
                                      forest_model.feature_importances_)),
               columns =['Transcript', 'Feature_Importance'])
    forest_importance = forest_importance.sort_values(by='Feature_Importance')
    forest_importance.to_csv(os.path.join(out_dir, f'{protein_of_interest}_RandomForest.csv'))